# JSON-LD Conversions

This notebook demonstrates to convert Resources to JSON-LD and vice-versa, and the different possibilities to asign a context to a Resource. Thus has the following priority:
1. use the context explicitly assigned to the resource i.e.`jane.context="http://shchema.org"`
2. use the context configured in the RdfModel if any
3. Let the store add a context (if it is a RDF-store)

This notebook can be used with one of the two stores: 
* The Demo Store, which is an in-memory store. Some features demonstrated in this notebook are not supported for this store. Do not use it for production.
* Nexus, which uses [Blue Bran Nexus](https://bluebrainnexus.io/) as a backend.

In [1]:
from kgforge.core import KnowledgeGraphForge

## Create the Forge using Demo Store and Demo Model

In [2]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

## Create the Forge using Nexus Store and RDF Model

To run the demo using Nexus, use the next three cells. Provide the Nexus token and the organization and project you want to use.

In [ ]:
import getpass

In [ ]:
token = getpass.getpass()

In [ ]:
bucket = "dke/kgforge_tests"

In [ ]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", bucket=bucket, token=token)

# Context

In [3]:
context = {
    "ex": "http://example.org/",
    "Person": "ex:Person",
    "Organization": "ex:Organization",
    "employer": "ex:employer",
    "name": "ex:name"
}

# Imports

In [4]:
import json

In [5]:
from kgforge.core import Resource

In [6]:
def pp(x): 
    if x is not None:
        print(json.dumps(x, indent=4))

## Resource to JSON-LD

### context in the Resource

In [7]:
jane = Resource(type="Person", name="Jane Doe")

In [8]:
jane.context = context

In [9]:
print(jane)

{
    context:
    {
        Organization: ex:Organization
        Person: ex:Person
        employer: ex:employer
        ex: http://example.org/
        name: ex:name
    }
    type: Person
    name: Jane Doe
}


In [10]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [11]:
pp(forge.as_jsonld(jane, form="compacted"))

{
    "@context": {
        "ex": "http://example.org/",
        "Person": "ex:Person",
        "Organization": "ex:Organization",
        "employer": "ex:employer",
        "name": "ex:name"
    },
    "@id": "file:///Users/agarcia/Developments/kgforge/examples/notebooks/getting-started/e60c1efd-08a7-4f1e-a377-5a68f58acdbb",
    "@type": "Person",
    "name": "Jane Doe"
}


In [12]:
 pp(forge.as_jsonld(jane))

{
    "@context": {
        "ex": "http://example.org/",
        "Person": "ex:Person",
        "Organization": "ex:Organization",
        "employer": "ex:employer",
        "name": "ex:name"
    },
    "@id": "file:///Users/agarcia/Developments/kgforge/examples/notebooks/getting-started/e60c1efd-08a7-4f1e-a377-5a68f58acdbb",
    "@type": "Person",
    "name": "Jane Doe"
}


In [13]:
pp(forge.as_jsonld(jane, store_metadata=True))

<action> _as_jsonld_one
<error> NotSupportedError: no available context in the metadata



### context available in the web

In [14]:
ivan = Resource(context="http://schema.org", type="Person", name="Ivan Tyron")

In [15]:
pp(forge.as_jsonld(ivan))

{
    "@context": "http://schema.org",
    "type": "Person",
    "name": "Ivan Tyron"
}


In [16]:
forge.register(ivan)

<action> _register_one
<succeeded> True


In [17]:
pp(forge.as_jsonld(ivan, form="expanded"))

{
    "@id": "file:///Users/agarcia/Developments/kgforge/examples/notebooks/getting-started/b1d266d3-c2fc-4cb3-9005-659af21e2de6",
    "@type": "http://schema.org/Person",
    "http://schema.org/name": "Ivan Tyron"
}


### context from the model

The configured model provides a default context that will be used to create resources that do not have context provided.

In [18]:
hana = Resource(type="Person", name="Hana Lloyd")

In [19]:
pp(forge.as_jsonld(hana))

{
    "@context": {
        "Person": "schema:Person",
        "Association": "prov:Association",
        "type": "rdf:type",
        "name": "schema:name",
        "agent": "prov:agent"
    },
    "@type": "Person",
    "name": "Hana Lloyd"
}


In [20]:
forge.register(hana)

<action> _register_one
<succeeded> True


In [21]:
pp(forge.as_jsonld(hana, form="expanded"))

{
    "@id": "file:///Users/agarcia/Developments/kgforge/examples/notebooks/getting-started/21e04e90-45d6-40ad-a87d-c70d257537bb",
    "@type": "schema:Person",
    "schema:name": "Hana Lloyd"
}


### context available in the store

It is possible to use a context that is only available in the Store, such as `https://bbp.neuroshapes.org`

In [22]:
karen = Resource(context="https://bbp.neuroshapes.org", type="Person", name="Karen Jones")

In [23]:
forge.register(karen)

<action> _register_one
<succeeded> True


In [24]:
print(karen)

{
    context: https://bbp.neuroshapes.org
    id: 0d8a26b6-6df9-417f-8a14-e893f3fad81e
    type: Person
    name: Karen Jones
}


Since this context is not locally resolvable the json-ld conversion will fail.

In [25]:
forge.as_jsonld(karen)

<action> _as_jsonld_one
<error> ValueError: https://bbp.neuroshapes.org is not resolvable



## JSON-LD to Resource

In [26]:
paul = Resource(context=context, type="Person", name="Paul Owens")

In [27]:
data = {
    "@context": context,
    "@type": "Person",
    "name": "Paul Owens",
}

In [28]:
resource = forge.from_jsonld(data)

In [29]:
resource == paul

True